In [ ]:
import toml

import numpy as np
import matplotlib.pyplot as plt

import pylinac

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pymedphys
from pymedphys._experimental.wlutz import reporting
from pymedphys._experimental.wlutz import main as _wlutz

In [ ]:
data_files = pymedphys.zip_data_paths('saturated-images.zip')

In [ ]:
collimator_angles = toml.load([item for item in data_files if item.suffix == '.toml'][0])
collimator_angles

In [ ]:
jpg_paths = {
    item.name: item for item in data_files if item.suffix == '.jpg'
}
jpg_paths

In [ ]:
edge_lengths = [20, 26]
penumbra = 2
bb_diameter = 8

algorithms = [
    "PyMedPhys",
    f"PyLinac v{pylinac.__version__}"
]

In [ ]:
algorithm = algorithms[0]

results = {}

for filename, full_image_path in jpg_paths.items():
    x, y, image = _wlutz.load_iview_image(full_image_path)
    icom_field_rotation = -collimator_angles[filename]

    field_centre, bb_centre = _wlutz.calculate(
        full_image_path,
        algorithm,
        bb_diameter,
        edge_lengths,
        penumbra,
        icom_field_rotation,
    )

    fig, axs = reporting.image_analysis_figure(
        x,
        y,
        image,
        bb_centre,
        field_centre,
        icom_field_rotation,
        bb_diameter,
        edge_lengths,
        penumbra,
    )

    axs[0, 0].set_title(algorithm)
    print(field_centre, bb_centre)
    
    results[filename] = {
        'centre': np.round(field_centre, 2).tolist(),
        'bb': np.round(bb_centre, 2).tolist()
    }
    
    plt.show()

In [ ]:
baseline = {
    "000059DB.jpg": {"centre": [-1.29, -3.49], "bb": [-0.46, -3.03]},
    "000059DC.jpg": {"centre": [-0.85, -3.25], "bb": [-0.47, -2.92]},
    "000059DD.jpg": {"centre": [-0.86, -3.07], "bb": [-0.53, -2.84]},
    "000059DE.jpg": {"centre": [-1.0, -2.9], "bb": [-0.63, -2.69]},
    "000059DF.jpg": {"centre": [-1.0, -2.67], "bb": [-0.76, -2.6]},
    "000059E0.jpg": {"centre": [-0.97, -2.49], "bb": [-0.76, -2.51]},
    "000059E1.jpg": {"centre": [-1.02, -2.34], "bb": [-0.77, -2.43]},
    "000059E4.jpg": {"centre": [-0.71, -1.9], "bb": [-0.72, -2.28]},
    "000059E5.jpg": {"centre": [-0.73, -1.8], "bb": [-0.78, -2.21]},
}

for key, result in results.items():
    for item in result.keys():
        assert np.allclose(result[item], baseline[key][item], atol=0.05)

In [ ]:
results